In [ ]:
from osgeo import gdal
import numpy as np
import pandas as pd
import csv
import os
import geopandas as gpd
from shapely.geometry import box

In [ ]:
def load_geotiff(file):
    ds = gdal.Open(file)
    band1 = ds.GetRasterBand(1).ReadAsArray() # Red channel
    band2 = ds.GetRasterBand(2).ReadAsArray() # Green channel
    band3 = ds.GetRasterBand(3).ReadAsArray() # Blue channel
    img = np.dstack((band1, band2, band3))
    return img

# Get image coordinates
def get_coordinates(f):
  ds = gdal.Open(f)
  gt = ds.GetGeoTransform()

  # print(ds.GetProjection())
  width = ds.RasterXSize
  height = ds.RasterYSize
  # print(width, height)

  minx = gt[0]
  miny = gt[3] + width*gt[4] + height*gt[5] 
  maxx = gt[0] + width*gt[1] + height*gt[2]
  maxy = gt[3]

  # Upper-left
  lon1, lat1 = minx, maxy
  # Bottom-right
  lon2, lat2 = maxx, miny

  return(lat1, lon1, lat2, lon2)

In [ ]:
FOLDER_PATH = 'folder_path_to_skysat_images'

out_file = open('bbox.csv', 'w')
writer = csv.writer(out_file)
header = ['filename', 'lat_1', 'lon_1', "lat_2", "lon_2"]
writer.writerow(header)

count = 0
for filename in os.listdir(FOLDER_PATH):
      f = os.path.join(FOLDER_PATH, filename)
      lat1, lon1, lat2, lon2 = get_coordinates(f)
      print(lat1, lon1, lat2, lon2)
      writer.writerow([filename, lat1, lon1, lat2, lon2])
      count += 1

      if count % 100 == 0:
            print(count)

out_file.close()